In [ ]:
Short_t NSINC = 5; 
TFile *file = new TFile(Form("../../Data/TSINC/T16N8R25L%droot%04d.root", NSINC, 4143)); 

TTree* fChain = (TTree*)file -> Get("tree"); 

TCanvas* c = new TCanvas(); 

In [ ]:
Short_t		hit = 0; 
Short_t		Ch[46]; 
Short_t		Cid[46]; 
Short_t		Sid[46]; 
Double_t	Evte[46]; 
Long64_t	Ts[46]; 
Double_t	Cfd[46]; 
Short_t		Index[46]; 
// Double_t	M[46]; 

In [ ]:
fChain -> SetBranchAddress("hit", &hit); 
fChain -> SetBranchAddress("cid", Cid); 
fChain -> SetBranchAddress("sid", Sid); 
fChain -> SetBranchAddress("ch", Ch); 
fChain -> SetBranchAddress("evte", Evte); 
fChain -> SetBranchAddress("ts", Ts); 
fChain -> SetBranchAddress("cfd", Cfd); 
fChain -> SetBranchAddress("index", Index); 
// fChain -> SetBranchAddress("M", M); 

In [ ]:
void ReadOut(std::ifstream &kbFile, Double_t *TimeLaBr){
	std::string line; 
	Int_t j = 0; 
	while (std::getline(kbFile, line))	{ //循环读取每一行
		Double_t Number; //创建整数变量
		std::stringstream ss(line); //将字符串转换为数据流
		for(Short_t i = 0; i < 9; i++){
			ss >> Number; 
			*(TimeLaBr + i + j * 9) = Number; //将数字添加到一维数组中
			std::cout<<Number<<"\t"; 
		}
		std::cout<<std::endl; 
		j += 1; 
	}
}

In [ ]:
Double_t	TimeLaBr[10][9]; 
std::ifstream kbFile(Form("Time%04d.txt", 4143)); // 打开文件
ReadOut(kbFile, TimeLaBr[0]); 

In [ ]:
TH2D* Energy_Energy = new TH2D("Energy_Energy", "Energy_Energy", 1600, 0, 1600, 1600, 0, 1600); 
TH1D* h1332_1173 = new TH1D("h1332_1173", "Time1332-1173", 400, -2000, 2000); 
TH1D* h1173_1332 = new TH1D("h_1173", "Time1332-1173", 400, -2000, 2000); 
TH1D* h511_1173 = new TH1D("h511_1173", "Time511-1173", 400, -2000, 2000); 
TH2D* hALL_1173 = new TH2D("hALL_1173", "TimeALL-1173", 1600, 0, 1600, 4000, -2000, 2000); 
TH1D* h1173 = new TH1D("h1173", "\\hbox{Ni在1173keV的开窗谱}", 1500, 0, 1500); 
TH1I* hEnergy = new TH1I("hEnergy", "hEnergy", 1000, 0, 6000); 
TH1I* hcfd = new TH1I("hcfd", "cfd", 2200, -100, 2100); 

In [ ]:
Long64_t Nentry = fChain -> GetEntries(); 
Double_t dT; 
for(Long64_t Ientry = 0; Ientry < Nentry; Ientry++){
	fChain -> GetEntry(Ientry); 
	for(Short_t j = 0; j < hit; j++)
	{
		if(Ch[j] == 1) hEnergy -> Fill(M[j]); 
		hcfd -> Fill(Cfd[j]); 
		for(Short_t i = 0; i < hit; i++)	if(Ch[i] > Ch[j])
		{
			Energy_Energy -> Fill(Evte[j], Evte[i]); 
			if((Evte[j] > 1160 && Evte[j] < 1190))
			{
				h1173 -> Fill(Evte[i]); 
				dT = Cfd[i] - Cfd[j] + (Ts[i] - Ts[j]) * 10000 + (Index[i] - Index[j]) * 2000 - TimeLaBr[Ch[i]][Ch[j]]; 
				hALL_1173 -> Fill(Evte[i], dT); 
				if(Evte[i] > 1320 && Evte[i] < 1350){
					h1332_1173 -> Fill(dT); 
				}
			}
			if((Evte[i] > 1160 && Evte[i] < 1190))
			{
				if(Evte[j] > 1320 && Evte[j] < 1350){
					dT = Cfd[i] - Cfd[j] + (Ts[i] - Ts[j]) * 10000 + (Index[i] - Index[j]) * 2000 - TimeLaBr[Ch[i]][Ch[j]]; 
					h1173_1332 -> Fill(dT); 
				}
			}
		}
	}
}

In [ ]:
//%jsroot
TF1 *f = new TF1("f", "gaus"); 
//h1332_1173 -> SetLineColor(kGreen);
h1332_1173 -> Fit(f, "", "", -200, 200); 
h1332_1173 -> Draw(); 
// h1173_1332 -> Draw(); 
c -> Draw(); 

In [ ]:
h1173_1332 -> Draw(); 
c -> Draw(); 

In [ ]:
hALL_1173->Draw("colz"); 
c -> Draw()

In [ ]:
hcfd -> Draw(); 
c -> Draw(); 

In [ ]:
// TFile* fff = new TFile("cfdTimeL.root", "update"); 
// fff -> WriteObject(hcfd, Form("hcfdL%02d", NSINC)); 
// fff -> Close(); 

In [ ]:
//%jsroot

hEnergy -> Draw(); 
hEnergy -> Fit("gaus", "", "", 5100, 5400); 
c -> Draw(); 

In [ ]:
// TFile* fffE = new TFile("EnergyN16.root", "update"); 
// fffE -> WriteObject(hEnergy, Form("hE%02d", NSINC)); 
// fffE -> Close(); 

In [ ]:
// //%jsroot
// TH1I *h = (TH1I*)hALL_1173->ProfileX(); 
// h -> Draw("HIST"); 
// c -> Draw(); 

In [ ]:
TGraphErrors *g = new TGraphErrors(); 
Double_t x[] = {0.5, 1, 1.5, 2, 4, 8, 20}; 
Double_t y[] = {142.853, 140.428, 140.778, 140.558, 140.604, 140.743, 140.829}; 
Double_t z[] = {0.52709, 0.499979, 0.503744, 0.50198, 0.502051, 0.503314, 0.50418}; 
for(Short_t i = 0; i < 7; i++)	g -> AddPoint(x[i], y[i]); 
for(Short_t i = 0; i < 7; i++)	g -> SetPointError(i, 0, z[i]); 

In [ ]:
g -> Draw("AP*"); 
g -> Draw("same"); 
c -> Draw(); 

In [ ]:
TFile *f = new TFile("cfdTime.root", "read"); 
Short_t List[7] = {1, 2, 3, 4, 8, 16, 40}; 
TCanvas *c = new TCanvas(); 

TH1I *h[8]; 

In [ ]:
h[7] = (TH1I*)f -> Get("XIAcfd"); 
h[7] -> SetLineColor(8); 
h[7] -> Draw(); 

for(Short_t i = 0; i < 7; i++)
{
	h[i] = (TH1I*)f -> Get(Form("hcfd%02d", List[i])); 
	h[i] -> SetLineColor(i + 1); 
	h[i] -> Draw("same"); 
}

In [ ]:
%jsroot
for(Short_t i = 1; i < 7; i++)
{
}
c -> Draw(); 

In [ ]:
TLegend *leg = new TLegend(0.1, 0.6, 0.2, 0.9); 
for(Short_t i = 0; i < 8; i++){leg -> AddEntry(h[i], Form("%s%02d", h[i] -> GetTitle(), List[i]), "l"); }
leg -> Draw(); 
c -> Draw(); 

In [ ]:
// c -> SaveAs("cfd.pdf"); 

In [ ]:
TGraphErrors *g = new TGraphErrors(); 
Double_t x[] = {10, 20, 25, 30, 40}; 
Double_t y[] = {206.48, 144.402, 140.829, 145.065, 169.515}; 
Double_t z[] = {1.60033, 0.541695, 0.503314, 0.54967, 0.871675}; 
for(Short_t i = 0; i < 5; i++)	g -> AddPoint(x[i], y[i]); 
for(Short_t i = 0; i < 5; i++)	g -> SetPointError(i, 0, z[i]); 